In [7]:
import heapq
import os

class BinaryTreeNode:
    
    def __init__(self, value, frequency):
        self.value = value
        self.frequency = frequency
        self.left = None
        self.right = None
        
    def __lt__(self, other):
        return self.frequency < other.frequency
    
    def __eq__(self, other):
        return self.frequency == other.frequency


class HuffmanCoding:
    
    def __init__(self, path):
        self.path = path
        self.__heap = []
        self.__codes = {}
        self.__reverseCodes = {}
        
    def __make_frequency_dict(self, text):
        freq_dict = {}
        
        for char in text:
            
            if char not in freq_dict:
                freq_dict[char] = 1
            else:
                freq_dict[char] += 1
                
        return freq_dict
    
    
    def __buildHeap(self, freq_dict):
        
        for key in freq_dict:
            
            node = BinaryTreeNode(key, freq_dict[key])
            heapq.heappush(self.__heap, node)
            
    
    def __buildTree(self):
        
        while len(self.__heap) > 1:
            node1 = heapq.heappop(self.__heap)
            node2 = heapq.heappop(self.__heap)

            newNode = BinaryTreeNode(None, node1.frequency + node2.frequency)
            newNode.left = node1
            newNode.right = node2

            heapq.heappush(self.__heap, newNode)
        return
    
    
    def __buildCodes(self):
        root = heapq.heappop(self.__heap)
        self.__buildCodesHelper(root, '')
        
    def __buildCodesHelper(self, root, currentBits):
        
        if root is None:
            return
        
        if root.value is not None:   #a leaf node
            self.__codes[root.value] = currentBits
            self.__reverseCodes[currentBits] = root.value
            return
        
        self.__buildCodesHelper(root.left, currentBits+'0')
        self.__buildCodesHelper(root.right, currentBits+'1')
        
    
    def __getEncoded(self, text):
        encodedText = ''
        for char in text:
            encodedText += self.__codes[char]
            
        return encodedText
    
    
    def __getPaddedEncodedText(self, encodedText):
        
        paddingAmount = 8 - (len(encodedText)%8)
        for i in range(paddingAmount):
            encodedText += '0'
        paddingInfo = "{0:08b}".format(paddingAmount)
        
        padded_encoding_text = paddingInfo + encodedText
        return padded_encoding_text
    
    
    def __getBytesArray(self, padded_encoding_text):
        array = []
        for i in range(0, len(padded_encoding_text), 8):
            byte = padded_encoding_text[i : i+8]
            array.append(int(byte, 2))
            
        return array
    
    
    def compress(self):
        
        #get file from path
        fileName, fileExtension = os.path.splitext(self.path)
        outputPath = fileName + '.bin'
        
        #read text from file
        with open(self.path, 'r+') as file, open(outputPath, 'wb') as output:
            text = file.read()
            text = text.rstrip()  #removes trailing spaces
        
            #make frequency dictionary using the text
            freq_dict = self.__make_frequency_dict(text)

            #construct the heap from the freq dict
            self.__buildHeap(freq_dict)

            #construct the binary tree from the heap
            self.__buildTree()

            #construct the codes from binary tree
            self.__buildCodes()

            #creating the encoded texts using the codes
            encodedText = self.__getEncoded(text)

            #pad this encoded text
            padded_encoding_text = self.__getPaddedEncodedText(encodedText)
            bytes_array = self.__getBytesArray(padded_encoding_text)
            finalBytes = bytes(bytes_array)
            output.write(finalBytes)
            
        print("Compressed")

        #return this binary file as output
        return outputPath
    
    
    def __removePadding(self, text):
        paddingInfo = text[:8]
        extraPadding = int(paddingInfo, 2)
        
        text_after_padding_removed = text[8 : -1*extraPadding]
        return text_after_padding_removed

    
    
    def __decodeText(self, text):
        
        decodedText = ""
        currentBits = ""
        
        for bit in text:
            currentBits += bit
            
            if currentBits in self.__reverseCodes:
                decodedText += self.__reverseCodes[currentBits]
                currentBits = ""
                
        return decodedText
    
    
    def decompress(self, inputPath):
        
        fileName, fileExtension = os.path.splitext(self.path)
        outputPath = fileName + '_decompressed' + '.txt'
        
        with open(inputPath, 'rb') as file, open(outputPath, 'w') as output:
            bitString = ""
            byte = file.read(1)
            
            while byte:
                byte = ord(byte)
                bits = "{0:08b}".format(byte)
                bitString += bits
                byte = file.read(1)
                
            actualText = self.__removePadding(bitString)
            decompressedText = self.__decodeText(actualText)
            output.write(decompressedText)
            
        return
    

path = 'C:/Users/Anirudh/Desktop/huffman.txt'
h = HuffmanCoding(path)
outputPath = h.compress()
h.decompress(outputPath)
        
        

Compressed


'0b101'